In [1]:
from datashop import *
from data_functions import *

import pprint as pp



import plotly.offline as pyo
import plotly.graph_objs as go
#from pmdarima.arima import auto_arima

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller,kpss,coint,bds,q_stat,grangercausalitytests,levinson_durbin
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARMA,ARMAResults,ARIMA,ARIMAResults

def chart_test_set(predictions,test):

    data = [go.Scatter(x=predictions.index,
                y=predictions, name = "Predicted"),
            go.Scatter(x=test.index,
                y=test,name='Actual')]
    fig = go.Figure(data)

    fig.show()

    mse_error = mean_squared_error(test,predictions)
    rmse_error = rmse(test,predictions)

    print("Mean Squared Error: {}".format(mse_error))
    print("Root Mean Squared Error: {}".format(rmse_error))



In [6]:
eia_dict = {        
        'DailyPrice':['DailyPrice','PET.RWTC.D','241335','%Y%m%d'],
        'WeeklyStocks':['WeeklyStocks','PET.WTTSTUS1.W','235081','%Y%m%d'],
        'ProductSupplied':['ProductSupplied', 'PET.WRPUPUS2.W','401676','%Y%m%d']
    }

dprice = eia_dict['DailyPrice']

price = EIA_Series(
    dprice[1], 
    name = dprice[0],
    date_format=dprice[3],
    scale=True,
    end='20201214'
    )

depot = Depot()

for key,val in eia_dict.items():
    feature = EIA_Series(
        val[1],
        name=val[0],
        date_format=val[3],
        scale=True
    )

    depot.ingest(feature)

df_dow = pd.read_csv('dowlatest.csv')

df_dow['Date'] = df_dow['Date']

df_dow['Date']=pd.to_datetime(
            df_dow['Date'],
            format = "%Y-%m-%d"
        )

scaler = MinMaxScaler()
df_dow['DOW_closing'] = scaler.fit_transform(df_dow[['Close']])

df_dow.set_index(df_dow['Date'],drop=True,inplace=True)        
df_dow.sort_index(ascending=True,inplace=True)  
df_dow = df_dow.asfreq(freq='D').fillna(method='ffill')

frame = depot.scaled.dropna()   #.loc[:'20200101']

frame = pd.merge_asof(
                frame,
                df_dow['DOW_closing'],
                right_index=True,
                left_index=True
                )

train_end = '20200701'
test_start = '20200702'
test_end = frame.index.max()

train = frame.loc[:train_end]
test = frame.loc[test_start:test_end]

exogs = ['scaled_WeeklyStocks','scaled_ProductSupplied','DOW_closing']

first_model = SARIMAX(
    train['scaled_DailyPrice'],
    exog=train[exogs],
    order=(2,1,2)
    ).fit()

predictions = first_model.predict(
    start=test_start,
    end=test_end ,
    exog = test[exogs],
    dynamic=False,typ='levels')

df_predicted = predictions.to_frame()
df_predicted.index.names = ['Date']
df_predicted.columns = ['Predicted']

import sqlite3
conn = sqlite3.connect(working_dir + '/data/energydash.db')


df_predicted.to_sql('Prediction',conn,if_exists='replace')

import sqlite3
conn = sqlite3.connect(working_dir + '/data/energydash.db')

df = pd.read_sql('SELECT * FROM Prediction LIMIT 10',con=conn)
df.head()

,scaled_DailyPrice,scaled_WeeklyStocks,scaled_ProductSupplied,DOW_closing
Date,,,,
2020-12-03,0.453289,0.814816,0.541628,0.989492
2020-12-04,0.456470,0.844065,0.549281,1.000000
2020-12-05,0.456470,0.844065,0.549281,1.000000
2020-12-06,0.456470,0.844065,0.549281,1.000000
2020-12-07,0.453673,0.844065,0.549281,0.993728


Mean Squared Error: 0.00029757790064651455
Root Mean Squared Error: 0.017250446389775385


In [27]:
import sqlite3
conn = sqlite3.connect(working_dir + '/data/energydash.db')

df = pd.read_sql('SELECT * FROM Prediction LIMIT 10',con=conn)
df.head()

,Date,Predicted
0,2020-07-02 00:00:00,0.421842
1,2020-07-03 00:00:00,0.422186
2,2020-07-04 00:00:00,0.422294
3,2020-07-05 00:00:00,0.422243
4,2020-07-06 00:00:00,0.426380


,index,0
0,2019-10-02 00:00:00,0.491178
1,2019-10-03 00:00:00,0.492203
2,2019-10-04 00:00:00,0.497268
3,2019-10-05 00:00:00,0.497057
4,2019-10-06 00:00:00,0.497093


In [26]:
df_predicted = predictions.to_frame()
df_predicted.index.names = ['Date']
df_predicted.columns = ['Predicted']

import sqlite3
conn = sqlite3.connect(working_dir + '/data/energydash.db')


df_predicted.to_sql('Prediction',conn,if_exists='replace')

import sqlite3
conn = sqlite3.connect(working_dir + '/data/energydash.db')

df = pd.read_sql('SELECT * FROM Prediction LIMIT 10',con=conn)
df.head()


,Date,Predicted
0,2020-07-02 00:00:00,0.421842
1,2020-07-03 00:00:00,0.422186
2,2020-07-04 00:00:00,0.422294
3,2020-07-05 00:00:00,0.422243
4,2020-07-06 00:00:00,0.426380
